# Sistema Inteligente de Búsqueda de Rutas - Metro de Medellín

## Actividad 2 - Búsqueda y Sistemas Basados en Reglas

Este notebook contiene **todo el código del sistema** desde cero. Puedes ejecutarlo completamente sin necesidad del archivo `.py`.

**Estudiante**: Nury Melissa Gil Valencia  
**Materia**: Inteligencia Artificial  
**Profesor**: Joaquín Sánchez

---

## Objetivo

Desarrollar un sistema inteligente que, a partir de una base de conocimiento escrita en reglas lógicas, encuentre la mejor ruta para moverse desde un punto A hasta un punto B en el sistema de transporte masivo local (Metro de Medellín).

---

## Estructura del Notebook

1. **Importar librerías necesarias**
2. **Definir tipos y estructuras de datos**
3. **Implementar la Base de Conocimiento con reglas lógicas**
4. **Implementar el Buscador de Rutas (A* y BFS)**
5. **Inicializar el sistema con datos del Metro de Medellín**
6. **Ejemplos prácticos y pruebas**


## 1. Importar Librerías Necesarias

Primero importamos las librerías estándar de Python que necesitamos.


In [53]:
# Importar librerías estándar de Python
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from enum import Enum
import heapq
from collections import deque
import math

print("✓ Librerías importadas correctamente")


✓ Librerías importadas correctamente


## 2. Definir Tipos y Estructuras de Datos

Definimos las estructuras básicas que representan el sistema de transporte.


In [54]:
# Tipo de transporte disponible
class TipoTransporte(Enum):
    """Tipos de transporte disponibles"""
    METRO = "metro"
    BUS = "bus"
    TRANSMILENIO = "transmilenio"
    METROBUS = "metrobus"

# Estructura para representar una estación
@dataclass
class Estacion:
    """Representa una estación del sistema de transporte"""
    nombre: str
    tipo: TipoTransporte
    coordenadas: Tuple[float, float]
    lineas: List[str]
    servicios: List[str] = None

    def __post_init__(self):
        if self.servicios is None:
            self.servicios = []

# Estructura para representar una conexión entre estaciones
@dataclass
class Conexion:
    """Representa una conexión entre estaciones"""
    origen: str
    destino: str
    tiempo: int  # en minutos
    costo: float  # en pesos
    linea: str
    tipo: TipoTransporte

# Estructura para representar una ruta encontrada
@dataclass
class Ruta:
    """Representa una ruta encontrada"""
    estaciones: List[str]
    tiempo_total: int
    costo_total: float
    transbordos: List[str]
    lineas_utilizadas: List[str]
    prioridad: int = 0
    
    def __lt__(self, other):
        """Para comparación en cola de prioridad"""
        return self.prioridad < other.prioridad

print("✓ Tipos y estructuras de datos definidos")


✓ Tipos y estructuras de datos definidos


## 3. Implementar la Base de Conocimiento

La base de conocimiento contiene las reglas lógicas del sistema experto y almacena toda la información sobre estaciones y conexiones.


In [55]:
class BaseConocimiento:
    """
    Base de conocimiento con reglas lógicas para el sistema de transporte
    """
    
    def __init__(self):
        self.estaciones: Dict[str, Estacion] = {}
        self.conexiones: List[Conexion] = []
        self.reglas: List[Dict] = []
        self._inicializar_conocimiento()
    
    def _inicializar_conocimiento(self):
        """Inicializa la base de conocimiento con reglas y datos del sistema"""
        
        # Regla 1: Si dos estaciones están en la misma línea, están conectadas directamente
        self.reglas.append({
            'nombre': 'conexion_misma_linea',
            'premisa': lambda e1, e2: self._misma_linea(e1, e2),
            'conclusion': lambda e1, e2: self._crear_conexion_directa(e1, e2)
        })
        
        # Regla 2: Si una estación es de transferencia, permite cambio de línea
        self.reglas.append({
            'nombre': 'transferencia_permitida',
            'premisa': lambda est: 'transferencia' in est.servicios,
            'conclusion': lambda est: True
        })
        
        # Regla 3: Si el tiempo es menor y el costo es razonable, la ruta es preferible
        self.reglas.append({
            'nombre': 'ruta_preferible',
            'premisa': lambda ruta: self._evaluar_ruta(ruta),
            'conclusion': lambda ruta: True
        })
        
        # Regla 4: Si es hora pico, el tiempo de espera aumenta
        self.reglas.append({
            'nombre': 'tiempo_hora_pico',
            'premisa': lambda hora: 6 <= hora <= 9 or 17 <= hora <= 20,
            'conclusion': lambda tiempo_base: tiempo_base * 1.3
        })
        
        # Regla 5: Si hay conexión directa sin transbordos, es mejor
        self.reglas.append({
            'nombre': 'sin_transbordos',
            'premisa': lambda ruta: len(ruta.transbordos) == 0,
            'conclusion': lambda ruta: ruta.prioridad + 10
        })
    
    def agregar_estacion(self, estacion: Estacion):
        """Agrega una estación a la base de conocimiento"""
        self.estaciones[estacion.nombre] = estacion
    
    def agregar_conexion(self, conexion: Conexion):
        """Agrega una conexión entre estaciones"""
        self.conexiones.append(conexion)
    
    def _misma_linea(self, e1: str, e2: str) -> bool:
        """Verifica si dos estaciones están en la misma línea"""
        if e1 not in self.estaciones or e2 not in self.estaciones:
            return False
        est1 = self.estaciones[e1]
        est2 = self.estaciones[e2]
        return bool(set(est1.lineas) & set(est2.lineas))
    
    def _crear_conexion_directa(self, e1: str, e2: str):
        """Crea una conexión directa entre estaciones de la misma línea"""
        if e1 not in self.estaciones or e2 not in self.estaciones:
            return
        est1 = self.estaciones[e1]
        est2 = self.estaciones[e2]
        lineas_comunes = set(est1.lineas) & set(est2.lineas)
        if lineas_comunes:
            linea = list(lineas_comunes)[0]
            tiempo = self._calcular_tiempo_estimado(est1, est2)
            conexion = Conexion(
                origen=e1,
                destino=e2,
                tiempo=tiempo,
                costo=2500,
                linea=linea,
                tipo=est1.tipo
            )
            if conexion not in self.conexiones:
                self.conexiones.append(conexion)
    
    def _calcular_tiempo_estimado(self, e1: Estacion, e2: Estacion) -> int:
        """Calcula tiempo estimado entre dos estaciones"""
        dist = math.sqrt(
            (e1.coordenadas[0] - e2.coordenadas[0])**2 +
            (e1.coordenadas[1] - e2.coordenadas[1])**2
        )
        return int(dist * 2)  # Aproximadamente 2 minutos por unidad de distancia
    
    def _evaluar_ruta(self, ruta) -> bool:
        """Evalúa si una ruta es preferible según las reglas"""
        return ruta.tiempo_total < 60 and ruta.costo_total < 10000
    
    def obtener_conexiones(self, estacion: str) -> List[Conexion]:
        """Obtiene todas las conexiones desde una estación"""
        return [c for c in self.conexiones if c.origen == estacion]
    
    def aplicar_reglas(self, contexto: Dict):
        """Aplica las reglas lógicas según el contexto"""
        resultados = []
        for regla in self.reglas:
            if regla['premisa'](contexto):
                resultado = regla['conclusion'](contexto)
                resultados.append({
                    'regla': regla['nombre'],
                    'resultado': resultado
                })
        return resultados

print("✓ Clase BaseConocimiento implementada")


✓ Clase BaseConocimiento implementada


## 4. Implementar el Buscador de Rutas

Implementamos los algoritmos de búsqueda: A* (búsqueda informada) y BFS (búsqueda no informada).


In [56]:
class BuscadorRutas:
    """
    Sistema de búsqueda de rutas usando algoritmos heurísticos
    """
    
    def __init__(self, base_conocimiento: BaseConocimiento):
        self.bc = base_conocimiento
    
    def heuristica(self, estacion_actual: str, destino: str) -> float:
        """
        Función heurística para estimar la distancia restante
        Usa distancia euclidiana entre coordenadas
        """
        if estacion_actual not in self.bc.estaciones or destino not in self.bc.estaciones:
            return float('inf')
        
        est_actual = self.bc.estaciones[estacion_actual]
        est_destino = self.bc.estaciones[destino]
        
        distancia = math.sqrt(
            (est_actual.coordenadas[0] - est_destino.coordenadas[0])**2 +
            (est_actual.coordenadas[1] - est_destino.coordenadas[1])**2
        )
        
        # Convertir distancia a tiempo estimado (minutos)
        return distancia * 2
    
    def buscar_ruta_a_estrella(self, origen: str, destino: str) -> Optional[Ruta]:
        """
        Algoritmo A* para encontrar la mejor ruta
        """
        if origen not in self.bc.estaciones or destino not in self.bc.estaciones:
            return None
        
        # Cola de prioridad: (f_score, g_score, estacion_actual, ruta_parcial)
        cola = [(0, 0, origen, [origen], 0, [])]  # f, g, estacion, ruta, costo, lineas
        visitados = set()
        mejor_ruta = None
        mejor_costo = float('inf')
        
        while cola:
            f_score, g_score, actual, ruta, costo_total, lineas = heapq.heappop(cola)
            
            if actual in visitados:
                continue
            
            visitados.add(actual)
            
            if actual == destino:
                # Calcular transbordos
                transbordos = []
                for i in range(len(lineas) - 1):
                    if lineas[i] != lineas[i + 1]:
                        transbordos.append(ruta[i + 1])
                
                ruta_encontrada = Ruta(
                    estaciones=ruta,
                    tiempo_total=int(g_score),
                    costo_total=costo_total,
                    transbordos=transbordos,
                    lineas_utilizadas=lineas,
                    prioridad=int(g_score + len(transbordos) * 5)
                )
                
                if g_score < mejor_costo:
                    mejor_costo = g_score
                    mejor_ruta = ruta_encontrada
                continue
            
            # Obtener conexiones desde la estación actual
            conexiones = self.bc.obtener_conexiones(actual)
            
            for conexion in conexiones:
                if conexion.destino in visitados:
                    continue
                
                nuevo_g = g_score + conexion.tiempo
                nuevo_costo = costo_total + conexion.costo
                nueva_ruta = ruta + [conexion.destino]
                nuevas_lineas = lineas + [conexion.linea]
                
                h = self.heuristica(conexion.destino, destino)
                nuevo_f = nuevo_g + h
                
                heapq.heappush(cola, (
                    nuevo_f,
                    nuevo_g,
                    conexion.destino,
                    nueva_ruta,
                    nuevo_costo,
                    nuevas_lineas
                ))
        
        return mejor_ruta
    
    def buscar_ruta_anchura(self, origen: str, destino: str) -> Optional[Ruta]:
        """
        Búsqueda en anchura (BFS) como alternativa
        """
        if origen not in self.bc.estaciones or destino not in self.bc.estaciones:
            return None
        
        cola = deque([(origen, [origen], 0, 0, [])])  # estacion, ruta, tiempo, costo, lineas
        visitados = set()
        
        while cola:
            actual, ruta, tiempo_total, costo_total, lineas = cola.popleft()
            
            if actual in visitados:
                continue
            
            visitados.add(actual)
            
            if actual == destino:
                transbordos = []
                for i in range(len(lineas) - 1):
                    if lineas[i] != lineas[i + 1]:
                        transbordos.append(ruta[i + 1])
                
                return Ruta(
                    estaciones=ruta,
                    tiempo_total=tiempo_total,
                    costo_total=costo_total,
                    transbordos=transbordos,
                    lineas_utilizadas=lineas,
                    prioridad=tiempo_total
                )
            
            conexiones = self.bc.obtener_conexiones(actual)
            for conexion in conexiones:
                if conexion.destino not in visitados:
                    cola.append((
                        conexion.destino,
                        ruta + [conexion.destino],
                        tiempo_total + conexion.tiempo,
                        costo_total + conexion.costo,
                        lineas + [conexion.linea]
                    ))
        
        return None
    
    def buscar_mejor_ruta(self, origen: str, destino: str, algoritmo: str = 'a_estrella') -> Optional[Ruta]:
        """
        Busca la mejor ruta usando el algoritmo especificado
        """
        if algoritmo == 'a_estrella':
            return self.buscar_ruta_a_estrella(origen, destino)
        elif algoritmo == 'anchura':
            return self.buscar_ruta_anchura(origen, destino)
        else:
            raise ValueError(f"Algoritmo desconocido: {algoritmo}")

print("✓ Clase BuscadorRutas implementada")


✓ Clase BuscadorRutas implementada


## 5. Inicializar el Sistema con Datos del Metro de Medellín

Ahora inicializamos el sistema con datos reales del Metro de Medellín, incluyendo todas las estaciones y conexiones.


In [ ]:
def inicializar_sistema_metro_medellin() -> BaseConocimiento:
    """
    Inicializa el sistema con datos reales del Metro de Medellín
    Basado en el conocimiento local del sistema de transporte masivo de Medellín
    """
    bc = BaseConocimiento()
    
    # Estaciones del Metro de Medellín - Línea A (Norte-Sur)
    estaciones_datos = [
        # Línea A del Metro (Niquía - La Estrella)
        ("Niquía", TipoTransporte.METRO, (0, 0), ['Línea A'], ['transferencia']),
        ("Bello", TipoTransporte.METRO, (0, 2), ['Línea A'], []),
        ("Madera", TipoTransporte.METRO, (0, 4), ['Línea A'], []),
        ("San Antonio", TipoTransporte.METRO, (0, 6), ['Línea A'], ['transferencia']),
        ("Alpujarra", TipoTransporte.METRO, (0, 8), ['Línea A'], []),
        ("Exposiciones", TipoTransporte.METRO, (0, 10), ['Línea A'], []),
        ("Industriales", TipoTransporte.METRO, (0, 12), ['Línea A'], []),
        ("Poblado", TipoTransporte.METRO, (0, 14), ['Línea A'], ['transferencia']),
        ("Aguacatala", TipoTransporte.METRO, (0, 16), ['Línea A'], []),
        ("Itagüí", TipoTransporte.METRO, (0, 18), ['Línea A'], ['transferencia']),
        ("Sabaneta", TipoTransporte.METRO, (0, 20), ['Línea A'], []),
        ("La Estrella", TipoTransporte.METRO, (0, 22), ['Línea A'], []),
        
        # Rutas integradas desde Niquía (Barbosa)
        ("Barbosa", TipoTransporte.BUS, (-2, 0), ['Ruta Integrada Barbosa'], []),
        
        # Rutas integradas desde Poblado (zonas de trabajo)
        ("El Poblado Centro", TipoTransporte.BUS, (2, 14), ['Ruta Integrada Poblado'], []),
        ("Lleras", TipoTransporte.BUS, (2, 15), ['Ruta Integrada Poblado'], []),
        ("Oviedo", TipoTransporte.BUS, (2, 16), ['Ruta Integrada Poblado'], []),
    ]
    
    # Agregar estaciones
    for nombre, tipo, coord, lineas, servicios in estaciones_datos:
        estacion = Estacion(nombre, tipo, coord, lineas, servicios)
        bc.agregar_estacion(estacion)
    
    # Crear conexiones del Metro Línea A (bidireccionales)
    # IMPORTANTE: Para viajes SOLO en Metro (sin tarifa integrada): tarifa única de $3.430 COP
    # (no se cobra por tramo, es una tarifa única independientemente de cuántas estaciones recorras)
    # Estas conexiones por tramo se usan solo para calcular tiempos de viaje
    estaciones_metro = ["Niquía", "Bello", "Madera", "San Antonio", "Alpujarra", "Exposiciones", 
                        "Industriales", "Poblado", "Aguacatala", "Itagüí", "Sabaneta", "La Estrella"]
    
    # Conexiones por tramo (solo para calcular tiempos, no costos)
    tramos_metro = [
        ("Niquía", "Bello", 3),
        ("Bello", "Madera", 2),
        ("Madera", "San Antonio", 3),
        ("San Antonio", "Alpujarra", 2),
        ("Alpujarra", "Exposiciones", 2),
        ("Exposiciones", "Industriales", 3),
        ("Industriales", "Poblado", 3),
        ("Poblado", "Aguacatala", 2),
        ("Aguacatala", "Itagüí", 3),
        ("Itagüí", "Sabaneta", 2),
        ("Sabaneta", "La Estrella", 3),
    ]
    
    # Inicializar lista de conexiones
    conexiones_datos = []
    
    # Crear conexiones directas entre todas las estaciones del Metro con tarifa única $3.430
    # Esto refleja que si viajas solo en Metro, pagas una tarifa única independientemente de la distancia
    for i, origen in enumerate(estaciones_metro):
        for destino in estaciones_metro[i+1:]:
            # Calcular tiempo total sumando los tramos
            tiempo_total = 0
            origen_idx = estaciones_metro.index(origen)
            destino_idx = estaciones_metro.index(destino)
            # Sumar los tiempos de los tramos entre origen y destino
            for j in range(min(origen_idx, destino_idx), max(origen_idx, destino_idx)):
                tiempo_total += tramos_metro[j][2]
            
            # Conexión directa con tarifa única de $3.430 para viajes solo en Metro
            conexiones_datos.append((origen, destino, tiempo_total, 3430, "Línea A - Solo Metro", TipoTransporte.METRO))
            conexiones_datos.append((destino, origen, tiempo_total, 3430, "Línea A - Solo Metro", TipoTransporte.METRO))
    
    # Agregar el resto de conexiones
    conexiones_datos.extend([
        # RUTA INTEGRADA: Barbosa → Poblado (directo con tarifa integrada)
        # Tarifa integrada $5.255 incluye: Bus Barbosa → Niquía + Metro Niquía → Poblado
        # Esta es la forma correcta: pagas una sola tarifa integrada que cubre todo
        ("Barbosa", "Poblado", 43, 5255, "Ruta Integrada Barbosa", TipoTransporte.BUS),
        ("Poblado", "Barbosa", 43, 5255, "Ruta Integrada Barbosa", TipoTransporte.BUS),
        
        # NOTA IMPORTANTE: No creamos conexiones de costo 0 desde Niquía hacia otras estaciones del Metro
        # porque eso daría a entender que el Metro es gratis. En realidad:
        # - Si vienes desde Barbosa con tarifa integrada ($5.255), la conexión directa Barbosa → Poblado
        #   ya incluye el Metro hasta cualquier estación, por lo que no necesitas conexiones adicionales.
        # - Si viajas solo en Metro desde Niquía (sin tarifa integrada), debes usar las conexiones
        #   de "Línea A - Solo Metro" con tarifa única $3.430 que ya están creadas arriba.
        
        # Rutas integradas desde Poblado (buses integrados)
        # Tarifa integrada: ~$3.890 (incluye Metro de vuelta + bus)
        ("Poblado", "El Poblado Centro", 5, 3890, "Ruta Integrada Poblado", TipoTransporte.BUS),
        ("El Poblado Centro", "Poblado", 5, 3890, "Ruta Integrada Poblado", TipoTransporte.BUS),
        ("Poblado", "Lleras", 4, 3890, "Ruta Integrada Poblado", TipoTransporte.BUS),
        ("Lleras", "Poblado", 4, 3890, "Ruta Integrada Poblado", TipoTransporte.BUS),
        ("Poblado", "Oviedo", 6, 3890, "Ruta Integrada Poblado", TipoTransporte.BUS),
        ("Oviedo", "Poblado", 6, 3890, "Ruta Integrada Poblado", TipoTransporte.BUS),
        
        # Bus NO integrado desde Poblado (opción más económica)
        # Solo pagas el costo del bus sin integración (~$3.400)
        ("Poblado", "El Poblado Centro", 5, 3400, "Bus No Integrado", TipoTransporte.BUS),
        ("El Poblado Centro", "Poblado", 5, 3400, "Bus No Integrado", TipoTransporte.BUS),
    ])
    
    # Agregar conexiones
    for origen, destino, tiempo, costo, linea, tipo in conexiones_datos:
        conexion = Conexion(origen, destino, tiempo, costo, linea, tipo)
        bc.agregar_conexion(conexion)
    
    return bc

# Inicializar el sistema
bc = inicializar_sistema_metro_medellin()
buscador = BuscadorRutas(bc)

print("✓ Sistema inicializado correctamente")
print(f"✓ Total de estaciones: {len(bc.estaciones)}")
print(f"✓ Total de conexiones: {len(bc.conexiones)}")


✓ Sistema inicializado correctamente
✓ Total de estaciones: 16
✓ Total de conexiones: 142


## 6. Buscar Ruta Completa hasta el Destino Final

Ahora busquemos la ruta completa desde Barbosa hasta El Poblado Centro (destino de trabajo), incluyendo el bus integrado final.


In [50]:
# Buscar ruta completa hasta el destino de trabajo
origen = "Barbosa"
destino = "El Poblado Centro"

print(f"Buscando ruta completa de '{origen}' a '{destino}'...")
print("=" * 60)

ruta = buscador.buscar_mejor_ruta(origen, destino, 'a_estrella')

if ruta:
    print(f"\n✓ Ruta completa encontrada:")
    print(f"\n  Recorrido completo:")
    for i, estacion in enumerate(ruta.estaciones, 1):
        # Marcar transbordos
        if estacion in ruta.transbordos:
            print(f"    {i}. {estacion} ⚠️ TRANSBORDO")
        else:
            print(f"    {i}. {estacion}")
    
    print(f"\n  Resumen del viaje:")
    print(f"    • Tiempo total: {ruta.tiempo_total} minutos ({ruta.tiempo_total/60:.1f} horas)")
    print(f"    • Costo total: ${ruta.costo_total:,.0f} COP")
    print(f"    • Número de transbordos: {len(ruta.transbordos)}")
    print(f"    • Estaciones de transbordo: {', '.join(ruta.transbordos)}")
    
    # Calcular costo por transbordo
    if ruta.transbordos:
        print(f"\n  Desglose de transbordos:")
        print(f"    1. En Niquía: Cambio de bus integrado a Metro")
        if len(ruta.transbordos) > 1:
            print(f"    2. En Poblado: Cambio de Metro a bus integrado")
else:
    print("✗ No se encontró ruta")


Buscando ruta completa de 'Barbosa' a 'El Poblado Centro'...

✓ Ruta completa encontrada:

  Recorrido completo:
    1. Barbosa
    2. Poblado ⚠️ TRANSBORDO
    3. El Poblado Centro

  Resumen del viaje:
    • Tiempo total: 48 minutos (0.8 horas)
    • Costo total: $8,655 COP
    • Número de transbordos: 1
    • Estaciones de transbordo: Poblado

  Desglose de transbordos:
    1. En Niquía: Cambio de bus integrado a Metro


## 7. Comparar Algoritmos: A* vs BFS

Comparemos los resultados de los dos algoritmos implementados para la misma ruta.


In [59]:
# Comparar algoritmos A* y BFS
origen = "Niquía"
destino = "Itagüí"

print(f"Comparando algoritmos para ruta: {origen} → {destino}")
print("=" * 60)

# Algoritmo A*
ruta_a_estrella = buscador.buscar_mejor_ruta(origen, destino, 'a_estrella')

# Algoritmo BFS
ruta_bfs = buscador.buscar_mejor_ruta(origen, destino, 'anchura')

print("\n--- Algoritmo A* (Búsqueda Informada) ---")
if ruta_a_estrella:
    print(f"  Ruta: {' → '.join(ruta_a_estrella.estaciones)}")
    print(f"  Tiempo: {ruta_a_estrella.tiempo_total} minutos")
    print(f"  Costo: ${ruta_a_estrella.costo_total:,.0f} COP")
    print(f"  Transbordos: {len(ruta_a_estrella.transbordos)}")

print("\n--- Algoritmo BFS (Búsqueda No Informada) ---")
if ruta_bfs:
    print(f"  Ruta: {' → '.join(ruta_bfs.estaciones)}")
    print(f"  Tiempo: {ruta_bfs.tiempo_total} minutos")
    print(f"  Costo: ${ruta_bfs.costo_total:,.0f} COP")
    print(f"  Transbordos: {len(ruta_bfs.transbordos)}")

print("\n--- Comparación ---")
if ruta_a_estrella and ruta_bfs:
    print(f"  Ambos algoritmos encontraron la misma ruta: {'Sí' if ruta_a_estrella.estaciones == ruta_bfs.estaciones else 'No'}")
    print(f"  Diferencia de tiempo: {abs(ruta_a_estrella.tiempo_total - ruta_bfs.tiempo_total)} minutos")
    print(f"  Diferencia de costo: ${abs(ruta_a_estrella.costo_total - ruta_bfs.costo_total):,.0f} COP")


Comparando algoritmos para ruta: Niquía → Itagüí

--- Algoritmo A* (Búsqueda Informada) ---
  Ruta: Niquía → Itagüí
  Tiempo: 23 minutos
  Costo: $3,430 COP
  Transbordos: 0

--- Algoritmo BFS (Búsqueda No Informada) ---
  Ruta: Niquía → Itagüí
  Tiempo: 23 minutos
  Costo: $3,430 COP
  Transbordos: 0

--- Comparación ---
  Ambos algoritmos encontraron la misma ruta: Sí
  Diferencia de tiempo: 0 minutos
  Diferencia de costo: $0 COP


## 8. Aplicar Reglas Lógicas del Sistema Experto

Veamos cómo funcionan las reglas lógicas del sistema experto.


In [60]:
# Probar reglas lógicas
print("=" * 60)
print("APLICACIÓN DE REGLAS LÓGICAS")
print("=" * 60)

# Regla 1: Verificar si dos estaciones están en la misma línea
print("\n--- Regla 1: Conexión en misma línea ---")
est1, est2 = "Niquía", "Bello"
resultado = bc._misma_linea(est1, est2)
print(f"¿{est1} y {est2} están en la misma línea? {resultado}")
if resultado:
    print(f"  ✓ Regla aplicada: Estas estaciones están conectadas directamente")

est1, est2 = "Barbosa", "Poblado"
resultado = bc._misma_linea(est1, est2)
print(f"\n¿{est1} y {est2} están en la misma línea? {resultado}")
if not resultado:
    print(f"  ✓ Regla aplicada: Estas estaciones requieren transbordo")

# Regla 2: Verificar estaciones de transferencia
print("\n--- Regla 2: Estaciones de transferencia ---")
estaciones_transferencia = [nombre for nombre, est in bc.estaciones.items() 
                            if 'transferencia' in est.servicios]
print(f"Estaciones de transferencia: {', '.join(estaciones_transferencia)}")
print(f"  ✓ Regla aplicada: Estas estaciones permiten cambio de línea sin costo adicional")


APLICACIÓN DE REGLAS LÓGICAS

--- Regla 1: Conexión en misma línea ---
¿Niquía y Bello están en la misma línea? True
  ✓ Regla aplicada: Estas estaciones están conectadas directamente

¿Barbosa y Poblado están en la misma línea? False
  ✓ Regla aplicada: Estas estaciones requieren transbordo

--- Regla 2: Estaciones de transferencia ---
Estaciones de transferencia: Niquía, San Antonio, Poblado, Itagüí
  ✓ Regla aplicada: Estas estaciones permiten cambio de línea sin costo adicional


## 9. Función Heurística

La función heurística ayuda al algoritmo A* a estimar qué tan lejos está el destino.


In [61]:
# Probar función heurística
print("=" * 60)
print("FUNCIÓN HEURÍSTICA")
print("=" * 60)
print("\nLa heurística estima la distancia restante usando coordenadas geográficas.\n")

ejemplos_heuristica = [
    ("Barbosa", "Poblado"),
    ("Niquía", "Bello"),
    ("Niquía", "Itagüí"),
    ("Niquía", "Niquía"),
]

for origen, destino in ejemplos_heuristica:
    h = buscador.heuristica(origen, destino)
    print(f"  Heurística {origen:20s} → {destino:20s}: {h:6.2f} minutos estimados")
    
print("\nNota: La heurística ayuda a A* a explorar primero las rutas más prometedoras.")


FUNCIÓN HEURÍSTICA

La heurística estima la distancia restante usando coordenadas geográficas.

  Heurística Barbosa              → Poblado             :  28.28 minutos estimados
  Heurística Niquía               → Bello               :   4.00 minutos estimados
  Heurística Niquía               → Itagüí              :  36.00 minutos estimados
  Heurística Niquía               → Niquía              :   0.00 minutos estimados

Nota: La heurística ayuda a A* a explorar primero las rutas más prometedoras.


## 10. Ejecutar Todas las Pruebas

Ejecutemos la suite completa de pruebas para validar que todo funciona correctamente.


In [62]:
# Ejecutar todas las pruebas
import subprocess
import sys

print("Ejecutando suite completa de pruebas...")
print("=" * 60)

resultado = subprocess.run([sys.executable, "pruebas.py"], 
                          capture_output=True, 
                          text=True)

print(resultado.stdout)

if resultado.returncode == 0:
    print("\n✓ Todas las pruebas pasaron exitosamente")
else:
    print(f"\n✗ Algunas pruebas fallaron (código: {resultado.returncode})")
    if resultado.stderr:
        print("\nErrores:")
        print(resultado.stderr)


Ejecutando suite completa de pruebas...

SUITE DE PRUEBAS - Sistema de Búsqueda de Rutas

PRUEBA 1: Búsqueda básica

Buscando ruta de 'Barbosa' a 'Poblado'...
[OK] Ruta encontrada: Barbosa -> Poblado
[OK] Tiempo: 43 minutos
[OK] Costo: $5,255 COP
[OK] Prueba exitosa

PRUEBA 2: Búsqueda con transbordos

Buscando ruta de 'Barbosa' a 'El Poblado Centro'...
[OK] Ruta encontrada: Barbosa -> Poblado -> El Poblado Centro
[OK] Transbordos: 1
[OK] Estaciones de transbordo: Poblado
[OK] Prueba exitosa

PRUEBA 3: Comparación de algoritmos

Buscando ruta de 'Niquía' a 'Itagüí'...

--- Algoritmo A* ---
Ruta: Niquía -> Itagüí
Tiempo: 23 minutos
Costo: $3,430 COP

--- Algoritmo BFS ---
Ruta: Niquía -> Itagüí
Tiempo: 23 minutos
Costo: $3,430 COP
[OK] Prueba exitosa

PRUEBA 4: Aplicación de reglas lógicas

--- Regla: Conexión misma línea ---
Niquía y Bello en misma línea: True
Barbosa y Poblado en misma línea: False

--- Regla: Estación de transferencia ---
Niquía es estación de transferencia: True
[OK

In [63]:
# Probar función heurística
print("=" * 60)
print("FUNCIÓN HEURÍSTICA")
print("=" * 60)
print("\nLa heurística estima la distancia restante usando coordenadas geográficas.\n")

ejemplos_heuristica = [
    ("Barbosa", "Poblado"),
    ("Niquía", "Bello"),
    ("Niquía", "Itagüí"),
    ("Niquía", "Niquía"),
]

for origen, destino in ejemplos_heuristica:
    h = buscador.heuristica(origen, destino)
    print(f"  Heurística {origen:20s} → {destino:20s}: {h:6.2f} minutos estimados")
    
print("\nNota: La heurística ayuda a A* a explorar primero las rutas más prometedoras.")


FUNCIÓN HEURÍSTICA

La heurística estima la distancia restante usando coordenadas geográficas.

  Heurística Barbosa              → Poblado             :  28.28 minutos estimados
  Heurística Niquía               → Bello               :   4.00 minutos estimados
  Heurística Niquía               → Itagüí              :  36.00 minutos estimados
  Heurística Niquía               → Niquía              :   0.00 minutos estimados

Nota: La heurística ayuda a A* a explorar primero las rutas más prometedoras.


## 12. Resumen del Sistema

### Componentes Implementados

1. **Sistema Experto**
   - Base de conocimientos con reglas lógicas
   - Motor de inferencia que aplica las reglas

2. **Algoritmos de Búsqueda**
   - **A*** (Búsqueda informada): Usa heurísticas para encontrar rutas óptimas
   - **BFS** (Búsqueda no informada): Explora sistemáticamente todas las posibilidades

3. **Reglas Lógicas**
   - Conexión en misma línea
   - Transferencias permitidas
   - Evaluación de rutas preferibles
   - Ajuste por hora pico
   - Priorización de rutas sin transbordos

### Conocimiento Local Aplicado

Este sistema está basado en el conocimiento real del Metro de Medellín:
- Estaciones reales de la Línea A
- Tarifas actuales (2025)
- Rutas integradas desde municipios como Barbosa
- Rutas integradas hacia zonas de trabajo como El Poblado

### Caso de Estudio Personal

**Ruta típica**: Barbosa → Poblado → El Poblado Centro
- Tiempo aproximado: 48 minutos
- Costo aproximado: $8,655 COP
  - Tarifa integrada Barbosa → Poblado: $5,255 COP (incluye bus + Metro)
  - Bus no integrado Poblado → El Poblado Centro: $3,400 COP
- Transbordos: 1 (en Poblado, cambio de Metro a bus)

**Nota importante sobre tarifas integradas**: 
La tarifa integrada de $5,255 desde Barbosa incluye tanto el bus desde Barbosa hasta Niquía como el Metro desde Niquía hasta cualquier estación (incluyendo Poblado). Por eso no se suma el costo del Metro por tramos cuando se usa tarifa integrada.

---

**Nota**: Este notebook es completamente independiente. Todo el código está incluido aquí y puede ejecutarse sin necesidad del archivo `.py`.

**Repositorio**: https://github.com/MelissaGilV/ACT_2__IA_IBERO


## 11. Resumen del Sistema

### Componentes Implementados

1. **Sistema Experto**
   - Base de conocimientos con reglas lógicas
   - Motor de inferencia que aplica las reglas

2. **Algoritmos de Búsqueda**
   - **A*** (Búsqueda informada): Usa heurísticas para encontrar rutas óptimas
   - **BFS** (Búsqueda no informada): Explora sistemáticamente todas las posibilidades

3. **Reglas Lógicas**
   - Conexión en misma línea
   - Transferencias permitidas
   - Evaluación de rutas preferibles
   - Ajuste por hora pico
   - Priorización de rutas sin transbordos

### Conocimiento Local Aplicado

Este sistema está basado en el conocimiento real del Metro de Medellín:
- Estaciones reales de la Línea A
- Tarifas actuales (2025)
- Rutas integradas desde municipios como Barbosa
- Rutas integradas hacia zonas de trabajo como El Poblado

### Caso de Estudio Personal

**Ruta típica**: Barbosa → Poblado → El Poblado Centro
- Tiempo aproximado: 48 minutos
- Costo aproximado: $8,655 COP
  - Tarifa integrada Barbosa → Poblado: $5,255 COP (incluye bus + Metro)
  - Bus no integrado Poblado → El Poblado Centro: $3,400 COP
- Transbordos: 1 (en Poblado, cambio de Metro a bus)

**Nota importante sobre tarifas integradas**: 
La tarifa integrada de $5,255 desde Barbosa incluye tanto el bus desde Barbosa hasta Niquía como el Metro desde Niquía hasta cualquier estación (incluyendo Poblado). Por eso no se suma el costo del Metro por tramos cuando se usa tarifa integrada.

---

**Repositorio**: https://github.com/MelissaGilV/ACT_2__IA_IBERO
